Load the .csv files, get the df/f and save it in a .npy. 

Save the raw data. 


Save the deconvoluted spike trains.


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage import io

from tools import moving_average, get_dff, bin_traces, get_spike_trains

In [2]:
# Recordings to use
recordings = {'1.young': ['20220627_14_41_37', '20220627_15_38_37', '20220627_15_56_22'],
'2.medium': ['20220922_15_18_53', '20220922_16_18_38', '20221006_13_26_06'],
'3.old': ['20220630_16_02_37']}

path_to_origin = '/Volumes/T7/organoids/for_paper/data/'

window_size = 5
bin_size = 5

In [8]:
# Go through each recording and save the dff
for age in recordings.keys():
    for recording in recordings[age]:
        print('Cleaning recording: ' + age + ' - ' + recording)

        # Load the video
        path_to_video = path_to_origin + age + '/' + recording + '/video.tif'
        video = io.imread(path_to_video)

        # Get projection
        projection = np.mean(video, axis=0)

        # Load the recording's results
        path_to_recording = path_to_origin + age + '/' + recording + '/Results.csv'
        df = pd.read_csv(path_to_recording)

        # Get the ROIs df
        rois_names = [col for col in df.columns if col.startswith('Mean')]
        traces = df[rois_names].values # time_points x rois 

        # Bin the traces
        b_traces = bin_traces(traces, bin_size)

        # Apply moving average
        smooth_traces = moving_average(b_traces, window_size)
        # Remove edges to avoid artifacts
        smooth_traces = smooth_traces[window_size:-window_size,:]

        # Get df/f
        dff = get_dff(smooth_traces)

        # Get the spike trains
        spike_trains = get_spike_trains(dff)

        # Save the dff
        path_to_save = path_to_origin + age + '/' + recording + '/dff.npy'
        np.save(path_to_save, dff)
        # Save the raw traces
        path_to_save = path_to_origin + age + '/' + recording + '/raw_traces.npy'
        np.save(path_to_save, traces)
        # Save the spike trains
        path_to_save = path_to_origin + age + '/' + recording + '/spike_trains.npy'
        np.save(path_to_save, spike_trains) 
        # Save the projection
        path_to_save = path_to_origin + age + '/' + recording + '/projection.npy'
        np.save(path_to_save, projection)

Cleaning recording: 1.young - 20220627_14_41_37
Cleaning recording: 1.young - 20220627_15_38_37
Cleaning recording: 1.young - 20220627_15_56_22
Cleaning recording: 2.medium - 20220922_15_18_53
Cleaning recording: 2.medium - 20220922_16_18_38
Cleaning recording: 2.medium - 20221006_13_26_06
Cleaning recording: 3.old - 20220630_16_02_37
